In [1]:
#Imports
import os,time,json
os.environ["OMP_NUM_THREADS"] = '1'
import pandas as pd
import numpy as np
import json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [2]:
with open('C:/Users/Mikey/Documents/keys/tmdb_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [3]:
tmdb.API_KEY=login['api-key']

# Folder for storage

In [4]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['akas_filtered.csv.gz', 'basics_filtered.csv.gz', 'ratings_filtered.csv.gz']

### Most of this code is from LP, some has been altered to appease my preferred methods for personal readability

In [5]:
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celia/tmdbsimple"""
    #Get Movie IDS
    movie=tmdb.Movies(movie_id)
    #Get Movie detailed info
    info=movie.info()
    #Movie Release info
    releases=movie.releases()
    #Iterate over the countires in releases data
    for c in releases['countries']:
        if c['iso_3166_1'] =='US':
            #Update cert
            info['certification'] = c['certification']
    return info

In [6]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    #Open Json file in read write
    with open(filename,'r+') as file:
        #Load exsiting file data
        file_data = json.load(file)
        #Check Check if new_data and file_data are lists.
        if isinstance(new_data, list) & isinstance(file_data, list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        #Move the file to beginning of file.        
        file.seek(0)
        #Write it
        json.dump(file_data, file)

In [7]:
## testing our function by looping through a list of test ids
test_ids = ["tt0848228", "tt0332280"]
results = []
errors = []
for movie_id in test_ids:
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
        
    except Exception as e: 
        errors.append([movie_id, e])
    
pd.DataFrame(results) 

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.709,28812,PG-13
1,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.880,10403,PG-13


In [8]:
#Custom Functions
basics=pd.read_csv(
'C:/Users/Mikey/Documents/GitHub/dojo-env-setup/Project-3/Project-3/Data/basics_filtered.csv.gz')
YEARS_TO_GET=[2000,2001]
errors=[]

In [9]:
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    #Json exists?
    if not file_exists:
        # If not, create new.
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id': 0}], f)
    #Select movies for current year
    df = basics.loc[basics['startYear'] == YEAR].copy()
    movie_ids = df['tconst'].copy()

    if file_exists:
        #If does, load previous data.
        previous_df = pd.read_json(JSON_FILE)
        #Get Movie ID's that have not processed.
        movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    else:
        #If doesnt exists get all movie ids for current year.
        movie_ids_to_get = movie_ids
    #Iterate over movie IDS to get movie details.    
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
                try:
                    # movie details with ratings.
                    temp = get_movie_with_rating(movie_id)
                    #Write to json
                    write_json(temp,JSON_FILE)
                    #Pause to avoid bombarding server.
                    time.sleep(0.02)
            
                except Exception as e:
                    #Store errors.
                    errors.append([movie_id, e])
    #Read final data for the year.                
    final_year_df = pd.read_json(JSON_FILE)
    #Save to a csv
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)
    #Print Error Count   
    print(f"- Total errors: {len(errors)}")

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1456 [00:00<?, ?it/s]

- Total errors: 208


Movies from 2001:   0%|          | 0/1575 [00:00<?, ?it/s]

- Total errors: 441
